In [5]:
import pandas as pd
import os

# 讀取資料
merged_data_path = 'merged_data.csv'

# 使用 pandas 讀取 CSV 檔案
merged_data = pd.read_csv(merged_data_path)

# 顯示合併後資料的前幾行，檢查欄位名稱
print(merged_data.head())

# 清理資料，只保留 "股票代號", "股票名稱" 和 "季底月均價" 欄位
# 先選擇包含的必要欄位
merged_data_cleaned = merged_data[['股票代號', '股票名稱','日期', '季底月均價']]  # 假設 'Unnamed: 8' 是 "季底月均價"
merged_data_cleaned.columns = ['股票代號', '股票名稱','日期', '季底月均價']

# 進一步清理資料，去除無效行
merged_data_cleaned = merged_data_cleaned.dropna(subset=['股票代號', '股票名稱','日期', '季底月均價'])

# 儲存整理後的資料到 CSV 檔案
output_file_path = 'cleaned_stock_data.csv'
merged_data_cleaned.to_csv(output_file_path, index=False)

# 顯示處理後的資料
import ace_tools as tools; tools.display_dataframe_to_user(name="清理後的股票資料", dataframe=merged_data_cleaned)

# 打印儲存路徑，便於下載
print(f"清理後的資料已儲存至: {output_file_path}")


          股票代號           股票名稱  \
0         股票名稱            NaN   
1          NaN            NaN   
2  Code & Name            NaN   
3     of Stock            NaN   
4        01     水泥工業類\nCement   

                                          Unnamed: 3    日期 Unnamed: 4  \
0  105.04.01 - 106.03.31.\nPrice Range \n01 Apr. ...   NaN        NaN   
1                                                NaN   NaN        NaN   
2                                             最高(元)    日期      最低(元)    
3                                            Highest  Date     Lowest   
4                                                NaN   NaN        NaN   

  Unnamed: 5                   Unnamed: 6                Unnamed: 7  \
0        NaN                        本月成交量                     本月成交值   
1        NaN                         （千股）                      （千元）   
2         日期  Trading Volume (1,000 Shs.)  Trading Value (NT$1,000)   
3       Date                          NaN                       NaN   
4      

C:\Users\Robby1206\AppData\Local\Temp\ipykernel_12408\76974011.py:8: DtypeWarning: Columns (0,11,12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv(merged_data_path)


ModuleNotFoundError: No module named 'ace_tools'

In [17]:
import pandas as pd

# 讀取資料
merged_data_path = 'cleaned_stock_data1.csv'
df = pd.read_csv(merged_data_path)

# 判斷字串是否是民國年格式（如 105/04/11）
def convert_roc_to_ad_safe(date_str):
    try:
        # 如果是已經是西元日期格式（看起來像 2019-07-02），就直接回傳
        if '-' in date_str:
            return date_str
        # 民國年轉換
        parts = date_str.split('/')
        roc_year = int(parts[0])
        ad_year = roc_year + 1911
        return f"{ad_year}/{parts[1]}/{parts[2]}"
    except Exception as e:
        print(f"❌ 無法解析日期: {date_str}, 錯誤：{e}")
        return pd.NaT  # 或 return None，視你要怎麼處理無法解析的值

# 套用轉換函數
df['日期_西元'] = df['日期'].apply(convert_roc_to_ad_safe)

# 轉換成 datetime
df['日期_西元'] = pd.to_datetime(df['日期_西元'], errors='coerce')  # 有錯就轉成 NaT

# 新增 'year_quarter' 欄位
df['year_quarter'] = df['日期_西元'].dt.to_period('Q').astype(str)

# 預覽結果
print(df[['日期', '日期_西元', 'year_quarter']])

# 匯出結果到新的 CSV 檔案
output_path = 'cleaned_stock_data_with_quarter.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')  # utf-8-sig 可避免中文亂碼
print(f"✅ 已成功儲存到：{output_path}")

                        日期      日期_西元 year_quarter
0                106/03/21 2017-03-21       2017Q1
1                106/03/22 2017-03-22       2017Q1
2                106/03/02 2017-03-02       2017Q1
3                106/02/17 2017-02-17       2017Q1
4                105/07/19 2016-07-19       2016Q3
...                    ...        ...          ...
14619  2020-02-27 00:00:00        NaT          NaT
14620  2020-01-20 00:00:00        NaT          NaT
14621  2020-02-03 00:00:00        NaT          NaT
14622  2020-08-18 00:00:00        NaT          NaT
14623  2020-07-08 00:00:00        NaT          NaT

[14624 rows x 3 columns]
✅ 已成功儲存到：cleaned_stock_data_with_quarter.csv


In [16]:
import pandas as pd

# 讀取資料（假設這是你給的格式）
df = pd.read_csv('all_mops_max_per_month.csv')

# 確保「月份」欄位為整數（若是字串先轉換）
df['月份'] = df['月份'].astype(int)

# 根據月份計算季度
def get_quarter(month):
    if 1 <= month <= 3:
        return 'Q1'
    elif 4 <= month <= 6:
        return 'Q2'
    elif 7 <= month <= 9:
        return 'Q3'
    elif 10 <= month <= 12:
        return 'Q4'
    else:
        return '錯誤'

# 新增 year_quarter 欄位
df['year_quarter'] = df['年度'].astype(str) + df['月份'].apply(get_quarter)

# 儲存新檔案
output_path = 'cleaned_stock_data_with_quarter.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 已成功產生新檔案：{output_path}")


✅ 已成功產生新檔案：cleaned_stock_data_with_quarter.csv


In [21]:
import pandas as pd
import numpy as np

# ========= Step 1：讀取三個檔案 =========
df_price = pd.read_csv('季底月均價.csv')
df_revenue = pd.read_csv('月營收_with_quarter.csv')
df_financial = pd.read_csv('財報_eps_data.csv')

# ========= Step 2：處理季底月均價 =========
df_price = df_price[['股票代號', '股票名稱', 'year_quarter', '季底月均價']]
df_price = df_price.rename(columns={'股票代號': '公司代號', '股票名稱': '公司名稱'})

# ========= Step 3：處理月營收 =========
df_revenue['月份'] = df_revenue['月份'].astype(int)
df_revenue_q = df_revenue[df_revenue['月份'].isin([3, 6, 9, 12])]

df_revenue_avg = df_revenue_q.groupby(['公司代號', '公司名稱', 'year_quarter'], as_index=False).agg({
    '營業收入-去年同月增減(%)': 'mean'
}).rename(columns={'營業收入-去年同月增減(%)': '營收成長_季平均'})

# ========= Step 4：處理財報資料 =========
df_financial = df_financial.rename(columns={
    '季度': 'year_quarter',
    '代碼': '公司代號',
    '名稱': '公司名稱',
    '營業收入': '財報_營業收入',
    '營業損益': '財報_營業損益',
    '業外收入': '財報_業外收入',
    '稅前損益': '財報_稅前損益',
    '稅後損益': '財報_稅後損益',
    '每股EPS': 'EPS'
})

# 去掉逗號和 '--' 並轉換成數值
num_cols = ['財報_營業收入', '財報_營業損益', '財報_業外收入', '財報_稅前損益', '財報_稅後損益', 'EPS']
for col in num_cols:
    df_financial[col] = df_financial[col].replace('--', np.nan)
    df_financial[col] = df_financial[col].replace(',', '', regex=True)
    df_financial[col] = pd.to_numeric(df_financial[col], errors='coerce')

# 新增財報比率特徵
df_financial['營業損益比'] = df_financial['財報_營業損益'] / df_financial['財報_營業收入']
df_financial['業外收入比'] = df_financial['財報_業外收入'] / df_financial['財報_稅前損益']

# ========= Step 5：統一 key 欄位的型別 =========
for df in [df_price, df_revenue_avg, df_financial]:
    df['公司代號'] = df['公司代號'].astype(str)
    df['公司名稱'] = df['公司名稱'].astype(str)
    df['year_quarter'] = df['year_quarter'].astype(str)

# ========= Step 6：合併資料 =========
# 合併財報 + 月營收
df_merge = pd.merge(df_financial, df_revenue_avg, on=['公司代號', '公司名稱', 'year_quarter'], how='left')

# 合併股價（季底均價）
df_merge = pd.merge(df_merge, df_price, on=['公司代號', '公司名稱', 'year_quarter'], how='left')

# ========= Step 7：儲存結果 =========
df_merge.to_csv('ml_merged_data.csv', index=False, encoding='utf-8-sig')
print("✅ 清整完成！已儲存為：ml_merged_data.csv")


✅ 清整完成！已儲存為：ml_merged_data.csv


In [22]:
import pandas as pd
import numpy as np

# ========= Step 1：讀取資料 =========
df_price = pd.read_csv('季底月均價.csv')
df_revenue = pd.read_csv('月營收_with_quarter.csv')
df_financial = pd.read_csv('財報_eps_data.csv')

# ========= Step 2：初步欄位清理與標準化 =========

# 標準化欄位名稱
df_price = df_price.rename(columns={'股票代號': '公司代號', '股票名稱': '公司名稱'})
df_price = df_price[['公司代號', '公司名稱', 'year_quarter', '季底月均價']]

df_financial = df_financial.rename(columns={
    '季度': 'year_quarter',
    '代碼': '公司代號',
    '名稱': '公司名稱',
    '營業收入': '財報_營業收入',
    '營業損益': '財報_營業損益',
    '業外收入': '財報_業外收入',
    '稅前損益': '財報_稅前損益',
    '稅後損益': '財報_稅後損益',
    '每股EPS': 'EPS'
})

# ========= Step 3：清理欄位空白與換行符 =========
for df in [df_price, df_revenue, df_financial]:
    df['公司代號'] = df['公司代號'].astype(str).str.strip()
    df['公司名稱'] = df['公司名稱'].astype(str).str.strip().str.replace('\n', '', regex=True)
    df['year_quarter'] = df['year_quarter'].astype(str).str.strip()

# ========= Step 4：清理財報數字欄位 =========
num_cols = ['財報_營業收入', '財報_營業損益', '財報_業外收入', '財報_稅前損益', '財報_稅後損益', 'EPS']
for col in num_cols:
    df_financial[col] = df_financial[col].replace('--', np.nan)
    df_financial[col] = df_financial[col].replace(',', '', regex=True)
    df_financial[col] = pd.to_numeric(df_financial[col], errors='coerce')

# 新增特徵
df_financial['營業損益比'] = df_financial['財報_營業損益'] / df_financial['財報_營業收入']
df_financial['業外收入比'] = df_financial['財報_業外收入'] / df_financial['財報_稅前損益']

# ========= Step 5：整理營收資料，取每季平均 =========
df_revenue['月份'] = df_revenue['月份'].astype(int)
df_revenue_q = df_revenue[df_revenue['月份'].isin([3, 6, 9, 12])]

df_revenue_avg = df_revenue_q.groupby(['公司代號', '公司名稱', 'year_quarter'], as_index=False).agg({
    '營業收入-去年同月增減(%)': 'mean'
}).rename(columns={'營業收入-去年同月增減(%)': '營收成長_季平均'})

# ========= Step 6：合併資料 =========
df_merge = pd.merge(df_financial, df_revenue_avg, on=['公司代號', '公司名稱', 'year_quarter'], how='left')
df_merge = pd.merge(df_merge, df_price, on=['公司代號', '公司名稱', 'year_quarter'], how='left')

# ========= Step 7：偵錯提示：哪些資料沒對到股價 =========
check_missing = df_merge[df_merge['季底月均價'].isnull()]
if not check_missing.empty:
    print("⚠️ 以下資料在『季底月均價.csv』中找不到對應的股價：")
    print(check_missing[['公司代號', '公司名稱', 'year_quarter']])
else:
    print("✅ 所有資料都成功對應到股價")

# ========= Step 8：輸出結果 =========
df_merge.to_csv('m2_merged_data.csv', index=False, encoding='utf-8-sig')
print("✅ 清整完成，已輸出到：ml_merged_data.csv")


⚠️ 以下資料在『季底月均價.csv』中找不到對應的股價：
       公司代號 公司名稱 year_quarter
0      1101   台泥       2017Q1
1      1102   亞泥       2017Q1
2      1103   嘉泥       2017Q1
3      1104   環泥       2017Q1
4      1108   幸福       2017Q1
...     ...  ...          ...
44391  8942   森鉅       2023Q2
44392  9949   琉園       2023Q2
44393  9950  萬國通       2023Q2
44394  9951   皇田       2023Q2
44395  9960  邁達康       2023Q2

[44396 rows x 3 columns]
✅ 清整完成，已輸出到：ml_merged_data.csv


In [ ]:
import pandas as pd
import numpy as np

# ========= Step 1：讀取三個檔案 =========
df_price = pd.read_csv('季底月均價.csv')
df_revenue = pd.read_csv('月營收_with_quarter.csv')
df_financial = pd.read_csv('財報_eps_data.csv')

# ========= Step 2：欄位清理 =========
# 統一欄位名稱
df_price = df_price.rename(columns={'股票代號': '公司代號', '股票名稱': '公司名稱'})
df_price = df_price[['公司代號', '公司名稱', 'year_quarter', '季底月均價']]

# ========= Step 3：資料欄位前處理（strip 空白與格式清理）=========
for df in [df_price, df_revenue, df_financial]:
    df['公司代號'] = df['公司代號'].astype(str).str.strip()
    df['公司名稱'] = df['公司名稱'].astype(str).str.strip().str.replace('\n', '', regex=True)
    df['year_quarter'] = df['year_quarter'].astype(str).str.strip()

# ========= Step 4：處理月營收資料 =========
df_revenue['月份'] = df_revenue['月份'].astype(int)

# 只取每季最後一個月（3、6、9、12）
df_revenue_q = df_revenue[df_revenue['月份'].isin([3, 6, 9, 12])]

# 計算每家公司每季的平均成長率
df_revenue_avg = df_revenue_q.groupby(['公司代號', '公司名稱', 'year_quarter'], as_index=False).agg({
    '營業收入-去年同月增減(%)': 'mean'
}).rename(columns={'營業收入-去年同月增減(%)': '營收成長_季平均'})

# ========= Step 5：處理財報資料與新增特徵 =========
df_financial = df_financial.rename(columns={
    '季度': 'year_quarter',
    '代碼': '公司代號',
    '名稱': '公司名稱',
    '營業收入': '財報_營業收入',
    '營業損益': '財報_營業損益',
    '業外收入': '財報_業外收入',
    '稅前損益': '財報_稅前損益',
    '稅後損益': '財報_稅後損益',
    '每股EPS': 'EPS'
})

# 處理數值欄位（移除 '--' 和 ','）
num_cols = ['財報_營業收入', '財報_營業損益', '財報_業外收入', '財報_稅前損益', '財報_稅後損益', 'EPS']
for col in num_cols:
    df_financial[col] = df_financial[col].replace('--', np.nan)
    df_financial[col] = df_financial[col].replace(',', '', regex=True)
    df_financial[col] = pd.to_numeric(df_financial[col], errors='coerce')

# 建立特徵欄位
df_financial['營業損益比'] = df_financial['財報_營業損益'] / df_financial['財報_營業收入']
df_financial['業外收入比'] = df_financial['財報_業外收入'] / df_financial['財報_稅前損益']

# ========= Step 6：合併資料 =========
# 合併 財報 + 月營收
df_merged = pd.merge(df_financial, df_revenue_avg, on=['公司代號', '公司名稱', 'year_quarter'], how='left')

# 合併 股價
df_merged = pd.merge(df_merged, df_price, on=['公司代號', '公司名稱', 'year_quarter'], how='left')

# ========= Step 7：印出找不到股價資料的紀錄 =========
missing_price = df_merged[df_merged['季底月均價'].isna()]
if not missing_price.empty:
    print("⚠️ 以下公司在該季度沒有股價資料（季底月均價）:")
    print(missing_price[['公司代號', '公司名稱', 'year_quarter']])
else:
    print("✅ 所有公司與季度都成功對到股價資料")

# ========= Step 8：輸出結果 =========
df_merged.to_csv('ml_merged_data.csv', index=False, encoding='utf-8-sig')
print("✅ 清整完成！已儲存為：ml_merged_data.csv")


In [23]:
import pandas as pd

# ========= Step 1：讀取原始股價檔案 =========
df_price = pd.read_csv('季底月均價.csv', quotechar='"', encoding='utf-8-sig')

# ========= Step 2：欄位標準化 =========
# 改欄位名稱，讓後面統一處理（可依實際欄位微調）
df_price = df_price.rename(columns={
    '股票代號': '公司代號',
    '股票名稱': '公司名稱',
    'year_quarter': 'year_quarter',
    '季底月均價': '季底月均價'
})

# ========= Step 3：清理資料內容 =========
# 去除空白、換行符號
df_price['公司代號'] = df_price['公司代號'].astype(str).str.strip()
df_price['公司名稱'] = df_price['公司名稱'].astype(str).str.replace('\n', '', regex=True).str.strip()
df_price['year_quarter'] = df_price['year_quarter'].astype(str).str.strip()

# 將季底月均價轉為 float，處理可能的逗號
df_price['季底月均價'] = df_price['季底月均價'].replace(',', '', regex=True)
df_price['季底月均價'] = pd.to_numeric(df_price['季底月均價'], errors='coerce')

# ========= Step 4：檢查欄位狀況 =========
print("✅ 清整後欄位：", df_price.columns.tolist())
print("✅ 資料筆數：", len(df_price))
print(df_price.head())

# ========= Step 5：輸出清整後的資料 =========
df_price.to_csv('cleaned_seasonal_price.csv', index=False, encoding='utf-8-sig')
print("✅ 檔案已儲存為：cleaned_seasonal_price.csv")


✅ 清整後欄位： ['公司代號', '公司名稱', '日期', '季底月均價', '日期_西元', 'year_quarter']
✅ 資料筆數： 14624
   公司代號              公司名稱         日期  季底月均價       日期_西元 year_quarter
0  1101           臺灣水泥TCC  106/03/21  37.82  2017-03-21       2017Q1
1  1102           亞洲水泥ACC  106/03/22  30.64  2017-03-22       2017Q1
2  1103           嘉新水泥CHC  106/03/02  11.53  2017-03-02       2017Q1
3  1104           環球水泥UCC  106/02/17  26.55  2017-02-17       2017Q1
4  1108  幸福水泥Lucky Cement  105/07/19  10.04  2016-07-19       2016Q3
✅ 檔案已儲存為：cleaned_seasonal_price.csv


In [1]:
import pandas as pd

# ========= Step 1：讀取資料 =========
df_price = pd.read_csv('季底月均價.csv', quotechar='"', encoding='utf-8-sig')

# ========= Step 2：欄位清理與統一 =========
# 重命名欄位為統一格式（可依你原始欄位名稱微調）
df_price = df_price.rename(columns={
    '股票代號': '公司代號',
    '股票名稱': '公司名稱',
    'year_quarter': 'year_quarter',
    '季底月均價': '季底月均價'
})

# 去除空白、換行符號、亂碼
df_price['公司代號'] = df_price['公司代號'].astype(str).str.strip()
df_price['公司名稱'] = df_price['公司名稱'].astype(str).str.replace('\n', '', regex=True).str.strip()
df_price['year_quarter'] = df_price['year_quarter'].astype(str).str.strip()

# 清理金額數字：把千分位逗號去掉
df_price['季底月均價'] = df_price['季底月均價'].replace(',', '', regex=True)
df_price['季底月均價'] = pd.to_numeric(df_price['季底月均價'], errors='coerce')

# ========= Step 3：依「公司代號 + 年季」做平均 =========
df_avg_price = df_price.groupby(['公司代號', '公司名稱', 'year_quarter'], as_index=False).agg({
    '季底月均價': 'mean'
}).rename(columns={'季底月均價': '季均價'})

# ========= Step 4：輸出清整後資料 =========
df_avg_price.to_csv('quarterly_avg_price.csv', index=False, encoding='utf-8-sig')
print("✅ 已完成每季平均股價清整，儲存為：quarterly_avg_price.csv")


✅ 已完成每季平均股價清整，儲存為：quarterly_avg_price.csv


In [3]:
import pandas as pd

# ========= Step 1：讀取原始檔案 =========
df_price = pd.read_csv('季底月均價.csv', quotechar='"', encoding='utf-8-sig')

# ========= Step 2：欄位清理 =========
df_price = df_price.rename(columns={
    '股票代號': '公司代號',
    'year_quarter': 'year_quarter',
    '季底月均價': '季底月均價'
})

# 清理公司代號與 year_quarter 欄位
df_price['公司代號'] = df_price['公司代號'].astype(str).str.strip()
df_price['year_quarter'] = df_price['year_quarter'].astype(str).str.strip()

# 清理數值欄位
df_price['季底月均價'] = df_price['季底月均價'].replace(',', '', regex=True)
df_price['季底月均價'] = pd.to_numeric(df_price['季底月均價'], errors='coerce')

# ========= Step 3：依公司代號 + 年季做平均 =========
df_avg_price = df_price.groupby(['公司代號', 'year_quarter'], as_index=False).agg({
    '季底月均價': 'mean'
}).rename(columns={'季底月均價': '季均價'})

# ========= Step 4：輸出結果 =========
df_avg_price.to_csv('quarterly_avg_price.csv', index=False, encoding='utf-8-sig')
print("✅ 季均價已計算完成，已儲存為：quarterly_avg_price.csv（不含公司名稱）")


✅ 季均價已計算完成，已儲存為：quarterly_avg_price.csv（不含公司名稱）


In [4]:
import pandas as pd
import numpy as np

# ========= Step 1：讀取原始財報資料 =========
df = pd.read_csv('財報_eps_data.csv', encoding='utf-8-sig')

# ========= Step 2：欄位重新命名（統一命名格式）=========
df = df.rename(columns={
    '季度': 'year_quarter',
    '代碼': '公司代號',
    '名稱': '公司名稱',
    '營業收入': '營業收入',
    '營業損益': '營業損益',
    '業外收入': '業外收入',
    '稅前損益': '稅前損益',
    '稅後損益': '稅後損益',
    '每股EPS': 'EPS'
})

# ========= Step 3：清洗數值欄位 =========
num_cols = ['營業收入', '營業損益', '業外收入', '稅前損益', '稅後損益', 'EPS']
for col in num_cols:
    df[col] = df[col].replace('--', np.nan)
    df[col] = df[col].replace(',', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# ========= Step 4：新增兩個特徵欄位 =========
df['營業利益率'] = df['營業損益'] / df['營業收入']
df['業外%'] = df['業外收入'] / df['稅前損益']

# ========= Step 5：輸出為新檔案 =========
output_path = 'financial_eps_with_ratios.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 已新增欄位：營業利益率、業外%")
print(f"📁 新檔案已儲存為：{output_path}")


✅ 已新增欄位：營業利益率、業外%
📁 新檔案已儲存為：financial_eps_with_ratios.csv


In [5]:
import pandas as pd

# ========= Step 1：讀取原始資料 =========
df = pd.read_csv('月營收_with_quarter.csv', encoding='utf-8-sig')

# ========= Step 2：保留每季最後一個月（3, 6, 9, 12） =========
df = df[df['月份'].isin([3, 6, 9, 12])]

# ========= Step 3：新增每個年季的平均營收增減率 =========
# 建立一個新的 DataFrame 儲存平均值
avg_growth = df.groupby(['公司代號', 'year_quarter'], as_index=False).agg({
    '營業收入-去年同月增減(%)': 'mean'
}).rename(columns={'營業收入-去年同月增減(%)': '營收成長_年季平均(%)'})

# ========= Step 4：將平均欄位合併回原始資料 =========
df = pd.merge(df, avg_growth, on=['公司代號', 'year_quarter'], how='left')

# ========= Step 5：只保留指定欄位 =========
df_filtered = df[[
    '公司代號', '公司名稱', '產業別', 'year_quarter',
    '營業收入-當月營收', '營收成長_年季平均(%)'
]]

# ========= Step 6：輸出結果 =========
output_path = 'quarterly_revenue_cleaned.csv'
df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"✅ 已儲存檔案為：{output_path}")


✅ 已儲存檔案為：quarterly_revenue_cleaned.csv
